<a href="https://colab.research.google.com/github/SammyDMartin/CC/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
#Uncomment for Colab
!pip install "ray[rllib]" torch
!pip install nashpy
!pip install pickle5
!git clone https://github.com/SammyDMartin/CC
"""

'\n#Uncomment for Colab\n!pip install "ray[rllib]" torch\n!pip install nashpy\n!pip install pickle5\n!git clone https://github.com/SammyDMartin/CC\n'

In [2]:
from cc_search import assess_agent_vs_others

ray version: 1.9.2


In [3]:
from cc_tools import DQNTorchPolicy,DQNTrainer,PPOTorchPolicy,PPOTrainer,A3CTorchPolicy,A3CTrainer
from cc_tools import maximin_policy,ideal_selfplay_policy,random_agent
from cc_tools import TFTAverage,RandomPolicy,IteratedPrisonersDilemma,IteratedBoS

from cc_search import get_rllib_config_eval
import numpy as np
from ray import tune

In [4]:
def reward_from_obs(obs_input,input_game):
    if len(obs_input) != 1:
        raise InterruptedError("Obs format is wrong, {}".format(obs_input))
    
    obs_batch = obs_input[0]

    if len(obs_batch) != 5:
        raise InterruptedError("MGSD specification is wrong, {}".format(obs_batch))
    
    act_number = int(np.where(obs_batch == 1)[0])
    if act_number == 4:
        return 0
    acts = [[0,0],[0,1],[1,0],[1,1]]
    action = acts[act_number]
    
    res= input_game[0]
    RC = input_game[1]

    if RC == 0:
        result = res[action[0],action[1]]
    elif RC == 1:
        result = res[action[1],action[0]]
    return result[RC]
    
    
    

In [23]:
from cc_search import BRToLastPolicy

In [36]:
tes = BRToLastPolicy([],[],{"SSD" : (IteratedBoS.PAYOUT_MATRIX,0)})

#tes.SSD_config[1]

tes.reward_from_obs_br_to_last(np.array([[0,0,0,0,1]]))

(0, 1)

In [ ]:
class LogPolicy_0(RandomPolicy):
    """Play the move that would beat the last move of the opponent."""
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        self.rew_memory = []
        self.act_memory = []
        self.obs_memory = []
        self.num = 1
        self.output = 0
        self.random_out = False
        self.SSD_config = config["SSD"]

    def update_target(self):
        pass
    def get_weights(self):
        pass
    def compute_actions(self,
                        obs_batch,
                        state_batches=None,
                        prev_action_batch=None,
                        prev_reward_batch=None,
                        **kwargs):
        input = obs_batch[0]
        self.obs_memory.append(obs_batch)
        self.rew_memory.append(prev_reward_batch)
        self.act_memory.append(prev_action_batch)

        output = self.output
        n = 6

        #print("Agent{}".format(self.num),state_batches,prev_action_batch,prev_reward_batch)
        #print("Agent{}".format(self.num),input,output,flush=True)

        rew_pred = reward_from_obs(obs_batch,self.SSD_config)

        """
        if len(self.obs_memory)>n+1:
            raise InterruptedError
            #pass

        if len(self.obs_memory)>n:
            print()
            for idx,_ in enumerate(self.obs_memory):
                obs, reward,acts = self.obs_memory[idx],self.rew_memory[idx],self.act_memory[idx]
                print("{}Agent{} {}: obs {}, reward {}={}, acts {}".format(self.SSD_config[1],self.num,idx,obs,reward,reward_from_obs(obs,self.SSD_config),acts))
        """

        #rew_pred = reward_from_obs(obs_batch,IteratedPrisonersDilemma.PAYOUT_MATRIX)
        rew = float(prev_reward_batch)

        assert (rew == rew_pred), "R:{}, Rpred{}, obs_batch{}".format(rew,rew_pred,obs_batch)

        if self.random_out == True:
            output = np.random.choice([0,1])


        return np.array([output]), \
                [], {}

class LogPolicy_1(LogPolicy_0):
    """Play the move that would beat the last move of the opponent."""
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        self.num = 2
        self.output = 1

class LogPolicy_rand(LogPolicy_0):
    """Play the move that would beat the last move of the opponent."""
    def __init__(self, observation_space, action_space, config):
        super().__init__(observation_space, action_space, config)
        self.num = 0
        self.output = None
        self.random_out = True

test_pol = lambda x,y: LogPolicy_rand

strategies = [test_pol]
opponents = []

In [ ]:
def test(agent1,agent2):
    rllib_config, stop_config = get_rllib_config_eval(
        seeds=[1],
        steps=5,
        game=IteratedBoS,
        debug=False,
        stop_iters=5,
        policy_classes=[agent1,agent2],
        checkpoint_paths=[]
    )

    rllib_config['env_config'].update({"output_additional_info":True})

    tune_analysis = tune.run(
        # Works with any Trainer but then using PGTrainer is more safe (simple training loop)... but PGTrainer doesn't work here for some reasons...
        DQNTrainer,
        # PPOTrainer,
        config=rllib_config,
        stop=stop_config,
        checkpoint_at_end=True, verbose = 0
    )

#test(LogPolicy_rand,LogPolicy_rand)

test(BRToLastPolicy,BRToLastPolicy)

In [ ]:
res1, res_full = assess_agent_vs_others(agent_trainer=(test_pol,None),opponents=[], strategies=[test_pol],steps=2)